In [1]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
trainfile = r'/gdrive/My Drive/508-A2/Fraud/Train.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/508-A2/Fraud/Test.csv'
testData = pd.read_csv(testfile) #creates a dataframe

#print sizes (shape) of datasets
print(trainData.shape)
print(testData.shape)

trainData.head()


(2999, 32)
(12918, 32)


,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY,FRAUDFOUND
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision,Yes
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils,Yes
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils,Yes
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils,Yes
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision,Yes


In [3]:
testData.head()

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY,FRAUDFOUND
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision,Yes
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils,Yes
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils,Yes
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils,Yes
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision,Yes


In [4]:
#Copy Train data excluding target
trainData_Copy = trainData.iloc[:, :-1].copy()
testData_Copy = testData.iloc[:, :-1].copy()

trainData_Copy.head()

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision


In [6]:
#List of Categorical Features
categoricalFeatures = ["MONTH", "DAYOFWEEK", 'MAKE', "ACCIDENTAREA", "DAYOFWEEKCLAIMED", 'MONTHCLAIMED', "SEX", 'MARITALSTATUS', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 
                       'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'BASEPOLICY']

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)

X_test.head()

,WEEKOFMONTH,WEEKOFMONTHCLAIMED,AGE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,YEAR,MONTH_Apr,MONTH_Aug,MONTH_Dec,MONTH_Feb,MONTH_Jan,MONTH_Jul,MONTH_Jun,MONTH_Mar,MONTH_May,MONTH_Nov,MONTH_Oct,MONTH_Sep,DAYOFWEEK_Friday,DAYOFWEEK_Monday,DAYOFWEEK_Saturday,DAYOFWEEK_Sunday,DAYOFWEEK_Thursday,DAYOFWEEK_Tuesday,DAYOFWEEK_Wednesday,MAKE_Accura,MAKE_BMW,MAKE_Chevrolet,MAKE_Dodge,MAKE_Ferrari,MAKE_Ford,MAKE_Honda,MAKE_Jaguar,MAKE_Lexus,MAKE_Mazda,MAKE_Mecedes,MAKE_Mercury,MAKE_Nisson,MAKE_Pontiac,...,AGEOFVEHICLE_2_years,AGEOFVEHICLE_3_years,AGEOFVEHICLE_4_years,AGEOFVEHICLE_5_years,AGEOFVEHICLE_6_years,AGEOFVEHICLE_7_years,AGEOFVEHICLE_more_than_7,AGEOFVEHICLE_new,AGEOFPOLICYHOLDER_16_to_17,AGEOFPOLICYHOLDER_18_to_20,AGEOFPOLICYHOLDER_21_to_25,AGEOFPOLICYHOLDER_26_to_30,AGEOFPOLICYHOLDER_31_to_35,AGEOFPOLICYHOLDER_36_to_40,AGEOFPOLICYHOLDER_41_to_50,AGEOFPOLICYHOLDER_51_to_65,AGEOFPOLICYHOLDER_over_65,POLICEREPORTFILED_No,POLICEREPORTFILED_Yes,WITNESSPRESENT_No,WITNESSPRESENT_Yes,AGENTTYPE_External,AGENTTYPE_Internal,NUMBEROFSUPPLIMENTS_1_to_2,NUMBEROFSUPPLIMENTS_3_to_5,NUMBEROFSUPPLIMENTS_more_than_5,NUMBEROFSUPPLIMENTS_none,ADDRESSCHANGE_CLAIM_1_year,ADDRESSCHANGE_CLAIM_2_to_3_years,ADDRESSCHANGE_CLAIM_4_to_8_years,ADDRESSCHANGE_CLAIM_no_change,ADDRESSCHANGE_CLAIM_under_6_months,NUMBEROFCARS_1-vehicle,NUMBEROFCARS_2-vehicles,NUMBEROFCARS_3_to_4,NUMBEROFCARS_5_to_8,NUMBEROFCARS_more_than_8,BASEPOLICY_All_Perils,BASEPOLICY_Collision,BASEPOLICY_Liability
0,3,4,21,4,400,4,1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
1,5,1,68,9,400,3,1994,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0
2,1,1,50,8,400,2,1994,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0
3,1,3,39,1,400,3,1994,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0
4,5,1,43,1,400,4,1994,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0


In [7]:
y_train = trainData["FRAUDFOUND"]

y_test = testData["FRAUDFOUND"]


y_test.head()


0    Yes
1    Yes
2    Yes
3    Yes
4    Yes
Name: FRAUDFOUND, dtype: object

In [9]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))


accuracy Score (training) for Decision Tree:0.881096
Confusion Matrix for Decision Tree
[[10935  1485]
 [   51   447]]


In [16]:
#Hyperparameter tuning done for decision tree classifier
#RANDOM SEARCH--------------------------------------------
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(5,95,10),'max_depth': range(10,60,10),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=20,cv=8)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#GRID SEARCH----------------------------------------`
print("GridSearchCV-Decision tree")
clf_grid = GridSearchCV(clf,parameters, cv = 8)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)



RandomizedSearchCV-Decision tree
{'min_samples_leaf': 35, 'max_depth': 30, 'criterion': 'entropy'}
GridSearchCV-Decision tree
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 35}


In [17]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clfr = DecisionTreeClassifier(**grid_parm1)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)


#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testing) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))

print("accuracy Score (testing) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')




accuracy Score (testing) after hypertuning randomized search for Decision Tree:0.907029
accuracy Score (testing) after hypertuning grid search for Decision Tree:0.907029
Confusion Matrix after hypertuning for Decision Tree
[[11568   852]
 [  349   149]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.97      0.93      0.95     12420
         Yes       0.15      0.30      0.20       498

    accuracy                           0.91     12918
   macro avg       0.56      0.62      0.57     12918
weighted avg       0.94      0.91      0.92     12918

[0.85480769 0.76153846 0.72884615 0.7        0.5875     0.5
 0.5        0.5        0.5375     0.6025641 ]




In [21]:
#Normal randomforest
rand_parameters={'min_samples_leaf' : range(5,95,10),'max_depth': range(1,20,2),'max_features':[10,12,15],'n_estimators':[25,35,45], 'criterion':['gini','entropy']}
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (testing) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))





accuracy Score (testing) for RandomForest:0.956882
Confusion Matrix for Random Forest:
[[11976   444]
 [  113   385]]


In [22]:
#RANDOMIZED SEARCH----------------------------------------
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=20,cv=8)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
rfc1= RandomForestClassifier(**grid_parm_rfc)
rfc1.fit(X_train,y_train)
rfc_predict = rfc1.predict(X_test)
print("accuracy Score (testing) after hypertuning for Random Forest:{0:6f}".format(rfc1.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
rfc_cv_score = cross_val_score(rfc, X_train, y_train)
print(rfc_cv_score)
print('\n')


{'n_estimators': 35, 'min_samples_leaf': 15, 'max_features': 15, 'max_depth': 17, 'criterion': 'gini'}
accuracy Score (testing) after hypertuning for Random Forest:0.947980
Confusion Matrix after hypertuning for Random Forest:
[[12162   258]
 [  414    84]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.97      0.98      0.97     12420
         Yes       0.25      0.17      0.20       498

    accuracy                           0.95     12918
   macro avg       0.61      0.57      0.59     12918
weighted avg       0.94      0.95      0.94     12918

[0.84       0.87       0.87187187]




In [23]:
print("GridSearchCV-Random Forest")
rfc_grid = GridSearchCV(rfc,rand_parameters,cv=8)
rfc_grid.fit(X_train, y_train)
grid_parm2=rfc_grid.best_params_
print(grid_parm2)

GridSearchCV-Random Forest
{'criterion': 'gini', 'max_depth': 19, 'max_features': 12, 'min_samples_leaf': 5, 'n_estimators': 35}


In [24]:
#Using the parameters obtained from HyperParameterTuning in the random forest 
rfc2 = RandomForestClassifier(**grid_parm2)

rfc2.fit(X_train,y_train)
rfc2_predict = rfc2.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testing) after hypertuning grid search for random forest:{0:6f}".format(rfc2.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for random forest")
print(confusion_matrix(y_test,rfc2_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc2_predict))

#clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="balanced_accuracy")
#print(clf_cv_score)
#print('\n')



accuracy Score (testing) after hypertuning grid search for random forest:0.938768
Confusion Matrix after hypertuning for random forest
[[11996   424]
 [  367   131]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.97      0.97      0.97     12420
         Yes       0.24      0.26      0.25       498

    accuracy                           0.94     12918
   macro avg       0.60      0.61      0.61     12918
weighted avg       0.94      0.94      0.94     12918

